<a href="https://colab.research.google.com/github/rybread1/trump-speech-writer2/blob/main/trump_speech_writer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime
import itertools
from collections import Counter
import re
import os

!pip install transformers

     |████████████████████████████████| 1.1MB 5.1MB/s 
     |████████████████████████████████| 3.0MB 20.8MB/s 
     |████████████████████████████████| 1.1MB 49.8MB/s 
     |████████████████████████████████| 890kB 47.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=21551a372c8643ee9da894645d62cb0b049dd2e4ba8302ef1ef8386862c24145
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
speeches_folder = '/gdrive/My Drive/colab-notebooks/trump-rally-speeches'
speeches_txt = []

for f in os.listdir(speeches_folder):
  f = os.path.join(speeches_folder, f)
  with open(f, 'r') as fp:
      txt = fp.read()
      txt = txt.strip()
      speeches_txt.append(txt)

In [6]:
from sklearn.model_selection import train_test_split

def build_text_files(txts, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for txt in txts:
        summary = txt.strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(speeches_txt, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 29
Test dataset length: 6


In [7]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [8]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):

    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=16)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=16)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,

    )
    return train_dataset,test_dataset,data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

In [9]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-trump", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:267: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  FutureWarning,


In [10]:
trainer.train()

{'loss': 3.583537353515625, 'learning_rate': 5e-05, 'epoch': 0.6613756613756614, 'total_flos': 191139545088000, 'step': 500}



{'loss': 3.272558837890625, 'learning_rate': 3.5859728506787336e-05, 'epoch': 1.3227513227513228, 'total_flos': 382207412846592, 'step': 1000}
{'loss': 3.14977490234375, 'learning_rate': 2.1719457013574662e-05, 'epoch': 1.9841269841269842, 'total_flos': 573346957934592, 'step': 1500}



{'loss': 2.9973662109375, 'learning_rate': 7.579185520361991e-06, 'epoch': 2.6455026455026456, 'total_flos': 764414825693184, 'step': 2000}




TrainOutput(global_step=2268, training_loss=3.220878802910053)

In [11]:
trainer.save_model()

In [25]:
from transformers import pipeline

trump = pipeline('text-generation', model='./gpt2-trump', tokenizer='gpt2', config={'max_length':10000, 'temperature': .5})

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [69]:
initial_seed = "The media is - really, the word, I think one of the greatest of all terms I've come up with - is fake."
# speech = initial_seed

# for i in range(10):
#   result = trump(initial_seed, max_length=100, do_sample=True, top_k=50)
#   speech += result[0]['generated_text'][len(initial_seed):]
#   initial_seed = result[0]['generated_text'][len(initial_seed):]

In [74]:
speech = trump(initial_seed, max_length=250, min_length=100, do_sample=True, temperature=1.2)[0]['generated_text']

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [75]:
import pprint

pprint.pprint(speech, width=160)

("The media is - really, the word, I think one of the greatest of all terms I've come up with - is fake. Can you say what you will? So what we do is we go out "
 'and say this. "Fake news". He\'ll be doing some crazy people, fake. We go out on social media from his base out and make up fake… But they\'ll only do it a '
 "few weeks later, not when they have the perfect candidate. But think of that. So now we're working on that, and we're working hard with other people for a "
 "couple of weeks. We'll have things done with a few of those people that we think are crazy, in real estate, some that I guess maybe just a fraction of the "
 'people that I actually know well. So now he\'s at stake. "Sir, your daughter Ivanka has died." They go, "Sir, you\'re an incredible golfer or a beautiful '
 'person, look, she was an incredible golfer at 18. A great golfing friend of mine, James Golan. She loved golf." The only thing that matters. I took her on '
 "the golf course. You've never seen anythin